<a href="https://colab.research.google.com/github/Ioncrade/OncoPath/blob/main/OT_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import torch
from torchvision.datasets import ImageFolder
from torch.utils.data import random_split, DataLoader
from torchvision import transforms, datasets
from transformers import ViTImageProcessor, ViTForImageClassification
from peft import LoraConfig, get_peft_model, TaskType
import patoolib


In [ ]:
!pip install patool

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.1/86.1 kB 5.4 MB/s eta 0:00:00


# Preprocessor

In [ ]:
processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224-in21k')

preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=processor.image_mean, std=processor.image_std)
])

## Data Loading

In [ ]:
patoolib.extract_archive("/content/data.zip")

INFO patool: Extracting /content/data.zip ...
INFO:patool:Extracting /content/data.zip ...
INFO patool: running /usr/bin/7z x -aou -o./Unpack_2d0d7xn4 -- /content/data.zip
INFO:patool:running /usr/bin/7z x -aou -o./Unpack_2d0d7xn4 -- /content/data.zip
INFO patool: ... /content/data.zip extracted to `data'.
INFO:patool:... /content/data.zip extracted to `data'.


'data'

In [ ]:
dataset_path = "/content/data"
train_dataset = datasets.ImageFolder(root=dataset_path, transform=transform)
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=4)

#Loading Model

In [ ]:
model = ViTForImageClassification.from_pretrained(
   "/content/drive/MyDrive/model.pt",
    num_labels=3,  # 3 classes: lung_scc, lung_n, lung_aca
    id2label={0: "lung_scc", 1: "lung_n", 2: "lung_aca"},
    label2id={"lung_scc": 0, "lung_n": 1, "lung_aca": 2}
)

##LoRA config

In [ ]:
lora_config = LoraConfig(
    task_type=TaskType.FEATURE_EXTRACTION,
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    target_modules=["query", "value"]
)
model = get_peft_model(model, lora_config)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
criterion = torch.nn.CrossEntropyLoss()

In [ ]:
from PIL import Image

In [ ]:
imageurl = '/content/image (1).webp'
image = Image.open(imageurl)

In [ ]:
inputs = processor(images=image, return_tensors="pt")
outputs = model(**inputs)
logits = outputs.logits

In [ ]:
predicted_class_idx = logits.argmax(-1).item()
print("Predicted class:", model.config.id2label[predicted_class_idx])

Predicted class: lung_aca
